In [ ]:
import pandas as pd
import numpy as np
import re
import spacy
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import string
# import matplotlib.pyplot as plt
# from wordcloud import WordCloud
# from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, DistilBertModel
import torch



In [ ]:
nlp = spacy.load('en_core_web_sm')

stopwords = ENGLISH_STOP_WORDS
# lemmatizer = WordNetLemmatizer()

def clean(doc):
    text_no_namedentities = []
    document = nlp(doc)
    ents = [e.text for e in document.ents]
    for item in document:
        if item.text in ents:
            pass
        else:
            text_no_namedentities.append(item.text)
    doc = (" ".join(text_no_namedentities))

    doc = doc.lower().strip()
    doc = doc.replace("</br>", " ")
    doc = doc.replace("-", " ")
    doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
    doc = " ".join([token for token in doc.split() if token not in stopwords])
    # doc = "".join([lemmatizer.lemmatize(word) for word in doc])
    doc=nlp(doc)
    doc = " ".join([word.lemma_ for word in doc])
    doc=" ".join(set((doc.split())))

    return doc

In [ ]:
def embedding(X_train):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    model = DistilBertModel.from_pretrained(model_name)
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)

    # X_train = job_description['job_description'].to_list()
    # print(type(X_train))

    res = tokenizer(X_train,padding=True, truncation=True, max_length=3, return_tensors="pt")
    # print(res)
    # len(res)

    with torch.no_grad():
        outputs = model(**res)
        # outputs = model(res['input_ids'])
        # model?
        JD_vects=outputs.last_hidden_state
    return JD_vects.reshape(1,-1)
    # return JD_vects[0].tolist()

In [ ]:
def cosim(df1,df2):
    return cosine_similarity(df1,df2)

In [ ]:
resume_data = pd.read_csv("resume_data.csv",index_col=0)


In [ ]:
# job_description = pd.read_csv('training_data.csv', chunksize=15)
job_description = pd.read_csv('training_data.csv')
job_description=job_description.head(15)


In [ ]:
resume_data['Cleaned_Resume']=resume_data['resume_text'].apply(lambda x:clean(x))


In [ ]:
job_description['Cleaned_JD']=job_description['job_description'].apply(lambda x:clean(x))


In [ ]:
resume_data['Cleaned_Resume_vector']=resume_data['Cleaned_Resume'].apply(lambda x:embedding(x))


In [ ]:
job_description['Cleaned_JD_vector']=job_description['Cleaned_JD'].apply(lambda x:embedding(x))


In [ ]:
choice = int(input("Enter index of a job decription to choose from 0-14:\n"))

if choice <= 14 and choice > 0:
    # job_description['company_name']
    print(f"\nYour choice of JD is from {job_description[job_description['company_name'].index==choice]['company_name'].values[0]}\n")
    print(f"Job Description:\n\n {job_description[job_description['company_name'].index==choice]['job_description'].values[0]}\n")
    print(f"Position for which resumes are shortlisted: {job_description[job_description['company_name'].index==choice]['position_title'].values[0]}\n")

    resume_data['jd_vector_of_choice'] = resume_data['Cleaned_Resume_vector'].apply(lambda x: cosim(x,job_description['Cleaned_JD_vector'][choice].tolist()))
    print("Shortlisted resume File names are as below\n")
    df=(resume_data.sort_values('jd_vector_of_choice', ascending=False)['File_name'].head())
    print(df)
    print()
else:
    print("Invalid choice try again")

Enter index of a job decription to choose from 0-14:
5

Your choice of JD is from DesignUps

Job Description:

 designups is a nashville based design and interactive agency at designups we have a strong focus on elevating brands with our expertise in design and customer experience

are you a frontend developer with a serious interest in design and ui this could be a great fit if so this job is for someone in the greater nashville area the majority of the work can be done remotely but we do require some time in our east nashville office location hey its not too bad to walk to five points pizza or wildcow for lunch

this is a contract position to start with a competitive hourly rate we would like to find the right person to join our team for a longterm role

skills needed
strong attention to design details typography ui etc
translating designs responsively for multiple screen sizes
familiarity with bootstrap or other similar frameworks
wordpress knowledge and desire to learn is a major b

In [ ]:
for each in df.to_list():
    print('Category: '+resume_data[['Category','File_name']][resume_data['File_name']==each].values[0][0]+'\n')
    print('File Name: '+resume_data[['Category','File_name']][resume_data['File_name']==each].values[0][1]+'\n')
    print('Resume Content: \n\n'+resume_data['resume_text'][resume_data['File_name']==each].values[0]+'\n')
    break

Category: ENGINEERING

File Name: 36149549.pdf

Resume Content: 

ENGINEERING PROJECT MANAGER
Summary
Seeking long term employment where my experience, expertise, and education can be used to satisfy the needs of the company, as well as provide
opportunity for personal development and career advancement. 
Project Coordination is my strongest point and I accept any challenges that I
encounter to get the job done. 
I offer no less than one hundred percent commitment to my employer, co-workers and my work. 
I am definitely a
team player but work well on my own with no supervision. 
I will bring to your company loyalty, dedication and great work ethics.
Highlights
Effective verbal and written communication and meeting facilitation skills
Proactive and adaptable self-starter
Self Motivated
Ability to prioritize and complete multiple tasks with little supervision
Punctual
Extroverted personality
Willing and available to work outside normal business, including weekends, to meet project deadli